# Chapter 9. Extensions

Plot the numerical results. Each part corresponds to a section of Chapter 9.

## Package imports and global variable definitions

In [ ]:
%pylab inline

In [ ]:
# uncomment this line if you prefer dynamic matplotlib plots
# %matplotlib notebook

# change the default figure size
# pylab.rcParams['figure.figsize'] = (10.0, 6.0)
pylab.rcParams['legend.fontsize'] = 12

In [ ]:
# manipulate dataframes
import pandas as pd

In [ ]:
# useful in the function dynamic
from scipy import special

In [ ]:
# global variables
ρρ = linspace(5., 0, 10000, endpoint = False)
nb_tokens = 6

## Combining job scheduling and load balancing

In [ ]:
# parameters
I = 10

In [ ]:
# load external results
dynamic_exact = pd.read_csv('../8-load-balancing/data/single-dynamic-exact.csv')
dynamic_exact_3 = pd.read_csv('data/single-dynamic-exact-3.csv')
dynamic_exact_4 = pd.read_csv('data/single-dynamic-exact-4.csv')
dynamic_exact_5 = pd.read_csv('data/single-dynamic-exact-5.csv')

In [ ]:
# average loss and activity probability
figure()

# dynamic
plot(dynamic_exact['rho'], dynamic_exact['beta'], 'C0', label="No parallelization")
plot(dynamic_exact['rho'], dynamic_exact['eta'], 'C0', label="")

# dynamic with 2 pooled computers
plot(dynamic_exact_3['rho'], dynamic_exact_3['beta'],
     'C1', label="Parallelization over 3 computers")
plot(dynamic_exact_3['rho'], dynamic_exact_3['eta'], 'C1', label="")

# dynamic with 2 pooled computers
plot(dynamic_exact_4['rho'], dynamic_exact_4['beta'],
     'C2', label="Parallelization over 4 computers")
plot(dynamic_exact_4['rho'], dynamic_exact_4['eta'], 'C2', label="")

# dynamic with 2 pooled computers
plot(dynamic_exact_5['rho'], dynamic_exact_5['beta'],
     'C5', label="Parallelization over 5 computers")
plot(dynamic_exact_5['rho'], dynamic_exact_5['eta'], 'C5', label="")

# ideal
plot(ρρ, maximum(0, 1. - 1. / ρρ), 'C3--', label="Ideal")
plot(ρρ, minimum(1, ρρ), 'C3--')

# references
axvline(x=2/5, color='C4', linestyle=':')
axvline(x=8/5, color='C4', linestyle=':')

xlim(0, 3); ylim(0, 1)
legend(loc = 'best')
xlabel("Load ρ"); ylabel("Job and computer metrics")
title("Average loss and activity probability")
show()

In [ ]:
# average loss and activity probability
figure()

# dynamic
semilogy(dynamic_exact['rho'], dynamic_exact['beta'], 'C0',
         label="No parallelization")

# dynamic with 3 pooled computers
semilogy(dynamic_exact_3['rho'], dynamic_exact_3['beta'], 'C1',
         label="Parallelization over 3 computers")

# dynamic with 4 pooled computers
semilogy(dynamic_exact_4['rho'], dynamic_exact_4['beta'], 'C2',
         label="Parallelization over 4 computers")

# dynamic with 5 pooled computers
semilogy(dynamic_exact_5['rho'], dynamic_exact_5['beta'], 'C5',
         label="Parallelization over 5 computers")

# ideal
semilogy(ρρ, maximum(0, 1. - 1. / ρρ), 'C3--', label="Ideal")

# reference
axvline(x=2/5, color='C4', linestyle=':')
axvline(x=8/5, color='C4', linestyle=':')

xlim(0.8, 1.2); ylim(0.001, 1)
legend(loc = 'best')
xlabel("Load ρ"); ylabel("Job and computer metrics")
title("Average loss and activity probability (I = " + str(I) + ")")
show()

In [ ]:
# mean number of jobs
figure()

# exact
plot(dynamic_exact['rho'], dynamic_exact['L'],
     'C0', label="No parallelization")
plot(dynamic_exact_3['rho'], dynamic_exact_3['L'],
     'C1', label="Parallelization over 3 computers")
plot(dynamic_exact_4['rho'], dynamic_exact_4['L'],
     'C2', label="Parallelization over 4 computers")
plot(dynamic_exact_5['rho'], dynamic_exact_5['L'],
     'C5', label="Parallelization over 5 computers")

# reference
axvline(x=2/5, color='C4', linestyle=':')
axvline(x=8/5, color='C4', linestyle=':')

xlim(0, 3); ylim(0, 10*nb_tokens)
legend(loc = 'best')
title("Mean number of jobs")
xlabel("Load ρ"); ylabel("Number of jobs")
show()

In [ ]:
# mean service rate
figure()

# exact
plot(dynamic_exact['rho'], dynamic_exact['gamma'], 'C0',
     label="No parallelization")
plot(dynamic_exact_3['rho'], dynamic_exact_3['gamma'], 'C1',
     label="Parallelization over 3 computers")
plot(dynamic_exact_4['rho'], dynamic_exact_4['gamma'], 'C2',
     label="Parallelization over 4 computers")
plot(dynamic_exact_5['rho'], dynamic_exact_5['gamma'], 'C5',
     label="Parallelization over 5 computers")

# reference
axvline(x=2/5, color='C4', linestyle=':')
axvline(x=8/5, color='C4', linestyle=':')
axhline(y=2.5/nb_tokens, color='C4', linestyle=':')

xlim(0, 3); ylim(ymin = 0)
legend(loc = 'best')
xlabel("Load ρ"); ylabel("Service rate")
title("Mean service rate")
show()

## Load balancing with multiple dispatchers

All numerical results are for two dispatchers only.

In [ ]:
def single_dynamic(l, μ, ν):
    # parameters
    barℓ = zeros(I, dtype=int)
    barℓ[1:] = cumsum(ℓ[:-1])
    
    # initialization: π_0
    π = ones(1, dtype=float64)
    
    for i in range(I):
        # recursion:  derive π_{i+1} from π_i
        
        # compute p_{i+1}
        p = ones(ℓ[i] + 1, dtype=float64)
        p[1:] = cumprod(μ[i] / (ν * arange(1, ℓ[i]+1)), dtype=float64)
        
        # make the outer product of π_i with p_{i+1}
        π = outer(π, p[::-1])
        
        # multiply each coefficient by t! / s!
        for s in range(barℓ[i] + 1):
            quotient = ones(ℓ[i] + 1, dtype=float64)
            quotient[1:] = cumprod(s + arange(1, ℓ[i] + 1), dtype=float64)
            π[s] *= quotient[::-1]
        
        # sum over the (anti)diagonals
        π = [trace(π, n) for n in range(ℓ[i], -barℓ[i] - 1, -1)]
    
    # normalize π
    #π /= sum(π)
    
    return 1. / sum(π)

### Static routing

In [ ]:
def static(l, α, β):
    maxl = max(l)
    cuml = cumsum(l)
    
    αs = ones(maxl+1, dtype=float64)
    αs[1:] = cumprod(α / arange(1, maxl+1))
    
    βt = ones(maxl+1, dtype=float64)
    βt[1:] = cumprod(β / arange(1, maxl+1))
    
    # i = 0
    oldG = outer(α**arange(l[0]+1), β**arange(l[0]+1)) * (tri(l[0]+1)[::-1])
    
    # i > 0
    for i in range(1, len(l)):
        G = zeros((cuml[i]+1, cuml[i]+1), dtype=float64)
        
        for ps in range(cuml[i-1]+1):
            for pt in range(cuml[i-1]-ps+1):
                prods = float64(1.)
                for s in range(ps, ps+l[i]+1):
                    prodt = float64(1.)
                    for t in range(pt, pt+l[i]-(s-ps)+1):
                        G[s,t] += oldG[ps, pt] * αs[s-ps] * βt[t-pt] * prods * prodt
                        prodt *= (t+1)
                    prods *= (s+1)
        
        oldG = copy(G)
    
    return sum(sum(G))

### Dynamic routing

In [ ]:
def dynamic(N, l, α, β):
    q = l // 2
    
    αs = ones(q+1, dtype=float64)
    αs[1:] = cumprod(α / arange(1, q+1))
    
    βt = ones(q+1, dtype=float64)
    βt[1:] = cumprod(β / arange(1, q+1))
    
    coeff = ones((q+1,q+1))
    for y in range(q+1):
        for z in range(q+1):
            coeff[y,z] = special.binom(l-y-z, q-y)
    
    # i = 0
    oldG = outer(α**arange(l+1), β**arange(l+1)) * (tri(l+1)[::-1])
    oldG = oldG[:q+1,:q+1] * coeff
    
    # i > 0
    for i in range(1, N):
        G = zeros((q*(i+1)+1, q*(i+1)+1), dtype=float64)
        
        for ps in range(q*i+1):
            for pt in range(min(q*i, l*i-ps) + 1):
                prods = float64(1.)
                for s in range(ps, ps + q + 1):
                    prodt = float64(1.)
                    for t in range(pt, pt + min(q,l-(s-ps)) + 1):
                        G[s,t] += oldG[ps, pt] * αs[s-ps] * βt[t-pt] * prods * prodt * coeff[s-ps,t-pt]
                        prodt *= (t+1)
                    prods *= (s+1)
        
        oldG = copy(G)
    
    return sum(sum(G))

### Balanced arrivals

In [ ]:
# parameters
ρρ = linspace(4,0,50, endpoint = False)
N = 10
ll = [2,4,6]
μ = 1.
α1 = .5 # proportion of arrivals at dispatcher 1
p1 = .5 # routing probabilities of tokens to dispatcher 1

In [ ]:
jiq_blocking = []
for l in ll:
    jiq_blocking.append([
        (
            (1-α1) / single_dynamic(l * ones(N, dtype=int), p1 * μ * ones(N), α1 * N * μ * ρ)
            + α1 / single_dynamic(l * ones(N, dtype=int), (1-p1) * μ * ones(N), (1-α1) * N * μ * ρ)
        )
        / static(l * ones(N, dtype=int), p1 / (α1*N*ρ), (1-p1) / ((1-α1)*N*ρ)) for ρ in ρρ
    ])

In [ ]:
baf_blocking = []
for l in ll:
    baf_blocking.append([
        (
            (1-α1) * dynamic(N, l, 1. / (α1*N*ρ), 0.)
            + α1 * dynamic(N, l, 0., 1. / ((1-α1)*N*ρ))
        )
        / dynamic(N, l, 1. / (α1*N*ρ), 1. / ((1-α1)*N*ρ))
        for ρ in ρρ
    ])

In [ ]:
# plot
for i,l in enumerate(ll):
    plot(ρρ, jiq_blocking[i], label = "JIQ - ℓ = " + str(l))

for i,l in enumerate(ll):
    plot(ρρ, baf_blocking[i], '--', color = 'C'+str(i), label = "BAF - ℓ = " + str(l))

#for i,l in enumerate(ll):
#    ρρρ, exp_blocking \
#    = loadtxt('data/dispatchers' + str(l) + '-balanced-blocking', delimiter='\t', unpack=True)
#    plot(ρρρ, exp_blocking, color = 'C' + str(i), linestyle="None", marker="x", label="JIQ - ℓ = " + str(l))

#for i,l in enumerate(ll):
#    ρρρ, exp_blocking \
#    = loadtxt('data/baf' + str(l) + '-balanced-blocking', delimiter='\t', unpack=True)
#    plot(ρρρ, exp_blocking, color = 'C' + str(i), linestyle="None", marker="+", label="BAF - ℓ = " + str(l))

xlim(0,4); ylim(0,1)
legend(loc = 4); title("Two dispatchers")
xlabel("Load ρ"); ylabel("Probability")
show()

In [ ]:
# save
csv = pd.DataFrame({'rho': ρρ})

for i,l in enumerate(ll):
    csv['jiq' + str(l)] = jiq_blocking[i]
    csv['baf' + str(l)] = baf_blocking[i]
    
fn = "data/theo-jiq-homo.csv"
csv.to_csv(fn, index = False)

### Imbalanced arrivals

In [ ]:
# parameters
ρρ = linspace(4,0,50, endpoint = False)
N = 10
ll = [2,4,6]
μ = 1.
α1 = .8 # proportion of arrivals at dispatcher 1
p1 = .5 # routing probabilities of tokens to dispatcher 1

In [ ]:
jiq_blocking = []
for l in ll:
    jiq_blocking.append([
        (
            (1-α1) / single_dynamic(l * ones(N, dtype=int), p1 * μ * ones(N), α1 * N * μ * ρ)
            + α1 / single_dynamic(l * ones(N, dtype=int), (1-p1) * μ * ones(N), (1-α1) * N * μ * ρ)
        )
        / static(l * ones(N, dtype=int), p1 / (α1*N*ρ), (1-p1) / ((1-α1)*N*ρ)) for ρ in ρρ
    ])

In [ ]:
baf_blocking = []
for l in ll:
    baf_blocking.append([
        (
            (1-α1) * dynamic(N, l, 1. / (α1*N*ρ), 0.)
            + α1 * dynamic(N, l, 0., 1. / ((1-α1)*N*ρ))
        )
        / dynamic(N, l, 1. / (α1*N*ρ), 1. / ((1-α1)*N*ρ))
        for ρ in ρρ
    ])

In [ ]:
# plot
for i,l in enumerate(ll):
    plot(ρρ, jiq_blocking[i], label = "JIQ - ℓ = " + str(l))

for i,l in enumerate(ll):
    plot(ρρ, baf_blocking[i], '--', color = 'C'+str(i), label = "BAF - ℓ = " + str(l))

#for i,l in enumerate(ll):
#    ρρρ, exp_blocking \
#    = loadtxt('data/dispatchers' + str(l) + '-imbalanced-blocking', delimiter='\t', unpack=True)
#    plot(ρρρ, exp_blocking, color = 'C' + str(i), linestyle="None", marker="x", label="JIQ - ℓ = " + str(l))

#for i,l in enumerate(ll):
#    ρρρ, exp_blocking \
#    = loadtxt('data/baf' + str(l) + '-imbalanced-blocking', delimiter='\t', unpack=True)
#    plot(ρρρ, exp_blocking, color = 'C' + str(i), linestyle="None", marker="+", label="BAF - ℓ = " + str(l))

xlim(0,4); ylim(0,1)
legend(loc = 4); title("Two dispatchers")
xlabel("Load ρ"); ylabel("Probability")
show()

In [ ]:
# save
csv = pd.DataFrame({'rho': ρρ})

for i,l in enumerate(ll):
    csv['jiq' + str(l)] = jiq_blocking[i]
    csv['baf' + str(l)] = baf_blocking[i]
    
fn = "data/theo-jiq-hete.csv"
csv.to_csv(fn, index = False)

### Imbalanced arrivals with adapted token routing

In [ ]:
# parameters
ρρ = linspace(4,0,50, endpoint = False)
N = 10
ll = [2,4,6]
μ = 1.
α1 = .8 # proportion of arrivals at dispatcher 1
p1 = .8 # routing probabilities of tokens to dispatcher 1

In [ ]:
jiq_blocking = []
for l in ll:
    jiq_blocking.append([
        (
            (1-α1) / single_dynamic(l * ones(N, dtype=int), p1 * μ * ones(N), α1 * N * μ * ρ)
            + α1 / single_dynamic(l * ones(N, dtype=int), (1-p1) * μ * ones(N), (1-α1) * N * μ * ρ)
        )
        / static(l * ones(N, dtype=int), p1 / (α1*N*ρ), (1-p1) / ((1-α1)*N*ρ)) for ρ in ρρ
    ])

In [ ]:
baf_blocking = []
for l in ll:
    baf_blocking.append([
        (
            (1-α1) * dynamic(N, l, 1. / (α1*N*ρ), 0.)
            + α1 * dynamic(N, l, 0., 1. / ((1-α1)*N*ρ))
        )
        / dynamic(N, l, 1. / (α1*N*ρ), 1. / ((1-α1)*N*ρ))
        for ρ in ρρ
    ])

In [ ]:
# plot
for i,l in enumerate(ll):
    plot(ρρ, jiq_blocking[i], label = "JIQ - ℓ = " + str(l))

for i,l in enumerate(ll):
    plot(ρρ, baf_blocking[i], '--', color = 'C'+str(i), label = "BAF - ℓ = " + str(l))

#for i,l in enumerate(ll):
#    ρρρ, exp_blocking \
#    = loadtxt('data/dispatchers' + str(l) + '-compensated-blocking', delimiter='\t', unpack=True)
#    plot(ρρρ, exp_blocking, color = 'C' + str(i), linestyle="None", marker="x", label="JIQ - ℓ = " + str(l))

#for i,l in enumerate(ll):
#    ρρρ, exp_blocking \
#    = loadtxt('data/baf' + str(l) + '-imbalanced-blocking', delimiter='\t', unpack=True)
#    plot(ρρρ, exp_blocking, color = 'C' + str(i), linestyle="None", marker="+", label="BAF - ℓ = " + str(l))

xlim(0,2); ylim(0,.6)
legend(loc = 4); title("Two dispatchers")
xlabel("Load ρ"); ylabel("Probability")
show()

### Homogeneous

In [ ]:
# parameters
ρρ = linspace(4,0,50, endpoint = False)
N = 10
l = 6
μ = 1.
α1 = .5 # proportion of arrivals at dispatcher 1

In [ ]:
p1 = .5 # routing probabilities of tokens to dispatcher 1

best_static = [ (
    (1-α1) / single_dynamic(l * ones(N, dtype=int), p1 * μ * ones(N), α1 * N * μ * ρ)
    + α1 / single_dynamic(l * ones(N, dtype=int), (1-p1) * μ * ones(N), (1-α1) * N * μ * ρ)
) / static(l * ones(N, dtype=int), p1 / (α1*N*ρ), (1-p1) / ((1-α1)*N*ρ)) for ρ in ρρ ]

In [ ]:
dynamic = [ (
    (1-α1) * dynamic(N, l, 1. / (α1*N*ρ), 0.)
    + α1 * dynamic(N, l, 0., 1. / ((1-α1)*N*ρ))
) / dynamic(N, l, 1. / (α1*N*ρ), 1. / ((1-α1)*N*ρ)) for ρ in ρρ ]

In [ ]:
# plot
plot(ρρ, dynamic, color = 'C0', label = "Dynamic")
plot(ρρ, ρρ * [1. - β for β in dynamic], color = 'C0')
plot(ρρ, best_static, '--', color = 'C1', label = "Best static = uniform_static static")
plot(ρρ, ρρ * [1. - β for β in best_static], color = 'C1')

#ρρρ, exp_blocking \
#= loadtxt('data/dispatchers' + str(l) + '-homo-dynamic-blocking', delimiter='\t', unpack=True)
#plot(ρρρ, exp_blocking, color = 'C0', linestyle="None", marker="+", label="Dynamic")

#ρρρ, exp_blocking \
#= loadtxt('data/dispatchers' + str(l) + '-homo-static-blocking', delimiter='\t', unpack=True)
#plot(ρρρ, exp_blocking, color = 'C1', linestyle="None", marker="x", label="Best static")

xlim(0,2); ylim(0,1)
legend(loc = 4); title("Two dispatchers")
xlabel("Load ρ"); ylabel("Probability")
show()

### Heterogeneous

In [ ]:
# parameters
ρρ = linspace(4,0,50, endpoint = False)
N = 10
l = 6
μ = 1.
α1 = .8 # proportion of arrivals at dispatcher 1

In [ ]:
p1 = .5 # routing probabilities of tokens to dispatcher 1

uniform_static_result = [ (
    (1-α1) / single_dynamic(l * ones(N, dtype=int), p1 * μ * ones(N), α1 * N * μ * ρ)
    + α1 / single_dynamic(l * ones(N, dtype=int), (1-p1) * μ * ones(N), (1-α1) * N * μ * ρ)
) / static(l * ones(N, dtype=int), p1 / (α1*N*ρ), (1-p1) / ((1-α1)*N*ρ)) for ρ in ρρ ]

In [ ]:
p1 = .8 # routing probabilities of tokens to dispatcher 1

best_static_result = [ (
    (1-α1) / single_dynamic(l * ones(N, dtype=int), p1 * μ * ones(N), α1 * N * μ * ρ)
    + α1 / single_dynamic(l * ones(N, dtype=int), (1-p1) * μ * ones(N), (1-α1) * N * μ * ρ)
) / static(l * ones(N, dtype=int), p1 / (α1*N*ρ), (1-p1) / ((1-α1)*N*ρ)) for ρ in ρρ ]

In [ ]:
dynamic_result = [ (
    (1-α1) * dynamic(N, l, 1. / (α1*N*ρ), 0.)
    + α1 * dynamic(N, l, 0., 1. / ((1-α1)*N*ρ))
) / dynamic(N, l, 1. / (α1*N*ρ), 1. / ((1-α1)*N*ρ)) for ρ in ρρ ]

In [ ]:
# plot
plot(ρρ, dynamic_result, color = 'C0', label = "Dynamic")
plot(ρρ, ρρ * [1. - β for β in dynamic_result], color = 'C0')
plot(ρρ, best_static_result, '--', color = 'C1', label = "Best static")
plot(ρρ, ρρ * [1. - β for β in best_static_result], color = 'C1')
plot(ρρ, uniform_static_result, '-.', color = 'C2', label = 'uniform_static static')
plot(ρρ, ρρ * [1. - β for β in uniform_static_result], color = 'C2')

#ρρρ, exp_blocking \
#= loadtxt('data/dispatchers' + str(l) + '-hete-dynamic-blocking', delimiter='\t', unpack=True)
#plot(ρρρ, exp_blocking, color = 'C0', linestyle="None", marker="x", label="Dynamic")

#ρρρ, exp_blocking \
#= loadtxt('data/dispatchers' + str(l) + '-hete-best_static-blocking', delimiter='\t', unpack=True)
#plot(ρρρ, exp_blocking, color = 'C1', linestyle="None", marker="+", label="Best static")

#ρρρ, exp_blocking \
#= loadtxt('data/dispatchers' + str(l) + '-hete-uniform_static-blocking', delimiter='\t', unpack=True)
#plot(ρρρ, exp_blocking, color = 'C2', linestyle="None", marker="x", label="uniform_static static")

xlim(0,4); ylim(0,1)
legend(loc = 4); title("Two dispatchers")
xlabel("Load ρ"); ylabel("Probability")
show()

In [ ]:
# save
csv = pd.DataFrame({'rho': ρρ,
                    'db': dynamic_result,
                    'do': ρρ * [1. - β for β in dynamic_result],
                    'ob': best_static_result,
                    'oo': ρρ * [1. - β for β in best_static_result],
                    'ub': uniform_static_result,
                    'uo': ρρ * [1. - β for β in uniform_static_result]
                    })

fn = "data/theo-jiq-comp.csv"
csv.to_csv(fn, index = False)